In [ ]:
pip install mp-api

In [ ]:
pip install mpcontribs-client

In [ ]:
pip install crystal-toolkit --upgrade

In [ ]:
pip install dash

http://henriquemiranda.github.io/phononwebsite/phonon.html?rest=https://materialsproject.org/rest/v2/materials/mp-66/phononbs&name=C&link=https://materialsproject.org/materials/mp-66&hidematlist=true

In [ ]:
from mp_api.client import MPRester
from pylab import *
import pandas as pd
from emmet.core.summary import HasProps
from pymatgen.io.xyz import XYZ
import py3Dmol
import nglview as nv
from nglview import show_structure_file
from pymatgen.io.cif import CifWriter

In [ ]:

material_id = "mp-996941"
API_key= "A55hSVu7NuwvlISilTvVtw37IB5ES26w"

with MPRester(API_key) as mpr:
    docs = mpr.summary.search(
        has_props = [HasProps.phonon], fields=["material_id", "formula_pretty","elements", "composition"]
    )
    mpids = [doc.material_id for doc in docs]
    mpformulas = [doc.formula_pretty for doc in docs]
    mpcomposition = [doc.composition for doc in docs]
data = {"mp ID": mpids, "Formula": mpformulas }
df_data = pd.DataFrame(data)
df_data


with MPRester(API_key) as mpr:
    ph_bs     = mpr.get_phonon_bandstructure_by_material_id(material_id)
    ph_dos    = mpr.get_phonon_dos_by_material_id(material_id)
    structure = mpr.get_structure_by_material_id(material_id)



position=[]
label= []
lines= []

for i in ph_bs.as_phononwebsite()["line_breaks"]:
    lines.append(i[0])


for i in ph_bs.as_phononwebsite()["highsym_qpts"]:
    position.append(i[0])
    label.append("$"+i[1]+ "$")


fig,ax = plt.subplots(ncols=2,nrows=1,figsize=(15,7),
                           gridspec_kw = {'wspace':0.2, 'hspace':0, 'width_ratios': [2, 1]})
ax[0].plot(ph_bs.as_phononwebsite()["eigenvalues"]);
density_points =len(ph_dos.as_dict()["densities"])
ax[0].set_ylim(0, (density_points));
ax[0].set_xlim(0, max(position ))
ax[0].set_xticks(position, label);

for i in lines:
    ax[0].axvline(i-1, c= "black")

ax[1].plot(ph_dos.as_dict()["densities"],arange(density_points))
ax[1].set_ylim(0, (density_points));
ax[1].set_xlim(0)

cif_filename = "material_structure_{0}.cif".format(material_id)  # Name of the output CIF file

cif_writer = CifWriter(structure)
cif_writer.write_file(cif_filename)
struct_file = nv.FileStructure(cif_filename)
view        = nv.show_file(struct_file)
view._remote_call("setSize", args=["","600px"])
view.camera = "orthographic"
view.add_unitcell()
view

In [ ]:
docs

In [ ]:
import json

# Sample dictionary
data = ph_bs.as_phononwebsite()

# Define the output JSON file name
output_filename = "data_{0}.json".format(material_id)

# Convert the dictionary to JSON and write to the file
with open(output_filename, "w") as json_file:
    json.dump(data, json_file, indent=4)

print(f"Dictionary has been converted and saved to {output_filename}")


In [ ]:
type()

# Working do not move

In [1]:
from ipywidgets import interact, Dropdown
import numpy as np
from pymatgen.io.cif import CifWriter
import nglview as nv
import pandas as pd
from pymatgen.ext.matproj import MPRester
from mp_api.client import MPRester
from pylab import *
import pandas as pd
from emmet.core.summary import HasProps

from nglview import show_structure_file
from pymatgen.io.cif import CifWriter

API_key= "A55hSVu7NuwvlISilTvVtw37IB5ES26w"

with MPRester(API_key) as mpr:
    docs = mpr.summary.search(
        has_props = [HasProps.phonon], fields=["material_id", "formula_pretty","elements", "composition"]
    )
    mpids = [doc.material_id for doc in docs]
    mpformulas = [doc.formula_pretty for doc in docs]
    mpcomposition = [doc.composition for doc in docs]
    

data = {"mp ID": mpids, "Formula": mpformulas}
df_data = pd.DataFrame(data)

d_look_up      = { mpformulas[i]:mpids[i] for i in range(len(mpformulas))}
d_look_up_comp = { mpformulas[i]:mpcomposition[i] for i in range(len(mpformulas))}

# Interactive function
@interact(material=Dropdown(options=sort(df_data["Formula"]), description="Materials available"))
def plot_phonon_and_structure(material):
    material_id = d_look_up[material][:]
    mat_comp    = d_look_up_comp[material]
    print("The ID of the material is: ", material_id)
    print("The composition of the material is: ", mat_comp)
    with MPRester(API_key) as mpr:
        ph_bs = mpr.get_phonon_bandstructure_by_material_id(material_id)
        ph_dos = mpr.get_phonon_dos_by_material_id(material_id)
        structure = mpr.get_structure_by_material_id(material_id)

    position = []
    label = []
    lines = []

    for i in ph_bs.as_phononwebsite()["line_breaks"]:
        lines.append(i[0])

    for i in ph_bs.as_phononwebsite()["highsym_qpts"]:
        position.append(i[0])
        label.append("$" + i[1] + "$")

    
    cif_filename = f"material_structure_{material_id}.cif"  # Name of the output CIF file

    cif_writer = CifWriter(structure)
    cif_writer.write_file(cif_filename)
    struct_file = nv.FileStructure(cif_filename)
    
    view = nv.show_file(struct_file)
    view._remote_call("setSize", args=["", "300px"])
    view.camera = "orthographic"
    view.add_unitcell()
    view

    
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15, 7), gridspec_kw={'wspace': 0.2, 'hspace': 0, 'width_ratios': [2, 1]})
    ax[0].plot(ph_bs.as_phononwebsite()["eigenvalues"])
    density_points = len(ph_dos.as_dict()["densities"])
    ax[0].set_ylim(0, (density_points))
    ax[0].set_xlim(0, max(position))
    ax[0].set_xticks(position, label)

    for i in lines:
        ax[0].axvline(i - 1, c="black")

    ax[1].plot(ph_dos.as_dict()["densities"], np.arange(density_points))
    ax[1].set_ylim(0, (density_points))
    ax[1].set_xlim(0)

    
    ph_ws_filename = "ph_ws_material_structure_{0}_{1}.json".format(material_id,material)
    ph_bs.write_phononwebsite(ph_ws_filename)
    return view

Retrieving SummaryDoc documents:   0%|          | 0/1507 [00:00<?, ?it/s]

interactive(children=(Dropdown(description='Materials available', options=('Ag2GeS3', 'Ag2HgI4', 'Ag2HgSI2', '…

In [ ]:
structure

# IR and raman

In [ ]:

material_id = "mp-2657"
API_key= "A55hSVu7NuwvlISilTvVtw37IB5ES26w"

with MPRester(API_key) as mpr:
    docs = mpr.summary.search(
        has_props = [HasProps.phonon], fields=["material_id", "formula_pretty","elements", "composition"]
    )
    mpids = [doc.material_id for doc in docs]
    mpformulas = [doc.formula_pretty for doc in docs]
    mpcomposition = [doc.composition for doc in docs]
data = {"mp ID": mpids, "Formula": mpformulas }
df_data = pd.DataFrame(data)
df_data


with MPRester(API_key) as mpr:
    ph_bs     = mpr.get_phonon_bandstructure_by_material_id(material_id)
    ph_bs_r     = mpr.get_phonon_bandstructure_by_material_id(material_id)
    ph_dos    = mpr.get_phonon_dos_by_material_id(material_id)
    structure = mpr.get_structure_by_material_id(material_id)


print(ph_bs.band_reorder())

In [ ]:
eigenvectors[2,2,5]

In [ ]:
import numpy as np

bands, qpoints, basis, vector = shape(ph_bs.eigendisplacements)
# Sample eigenvectors and eigenvalues (replace with your actual data)

eigenvectors = ph_bs.eigendisplacements
eigenvalues  = ph_bs.bands
qpoints_v = ph_bs.as_dict()["qpoints"]



# Constants (you might need to adjust these)
proportionality_IR = 1.0
proportionality_Raman = 1.0

# Calculate IR intensities
IR_intensities = np.zeros((bands, qpoints))
for i in range(bands):
    for q in range(qpoints):
        for basis_dir in range(basis):
            IR_intensities[i, q] += np.abs(np.dot(
                eigenvectors[i, q, basis_dir],
                qpoints_v[q])  # Replace with actual Cartesian direction vector
            ) ** 2

# Calculate Raman intensities
Raman_intensities = np.zeros((bands, qpoints))
for i in range(bands):
    # for q in range(qpoints):
    for basis_dir_alpha in range(basis):
        for basis_dir_beta in range(basis):
            Raman_intensities[i, 0] += np.abs(np.dot(
                        eigenvectors[i, 0, basis_dir_alpha],
                        eigenvectors[i, 0, basis_dir_beta]
                    )) ** 2 * proportionality_Raman

# Normalize intensities
IR_intensities *= proportionality_IR
Raman_intensities *= proportionality_Raman

# Now you can plot the IR and Raman spectra using your preferred plotting library
# (e.g., Matplotlib)
# Plot Raman Spectrum
plt.figure(figsize=(10, 6))
frequencies=eigenvalues
IR_total= list(zeros(len(frequencies[0])))
Freq_total= list(zeros(len(frequencies[0])))
for i in range(bands):
    plt.scatter(frequencies[i]*33.356, Raman_intensities[i], label=f'Mode {i+1}')
    IR_total+=list(Raman_intensities[i])
    Freq_total+=list(frequencies[i]*33.356)

plt.xlabel('Frequency (cm$^{-1}$)')
plt.ylabel('Intensity')
plt.title('Raman Spectrum')
plt.legend()
plt.grid()
plt.show()

Freq_0 = list(arange(0,2000,0.1))
IR_0   = list(zeros(len(Freq_0)))

Freq_total_s = Freq_0 + Freq_total
IR_total_s   = IR_0 + IR_total
all= []
for i in range(len(IR_total_s)):
    all.append(tuple((Freq_total_s[i],IR_total_s[i])))

all.sort()
x= []
y= []
for i in all:
    x.append(round(i[0],1))
    y.append(i[1])
    

In [ ]:
frequencies = x 
intensities = y 


# Crear un diccionario para realizar un seguimiento de las sumas de intensidades por frecuencia
frequency_intensity_dict = {}


for frequency, intensity in zip(frequencies, intensities):

    if frequency in frequency_intensity_dict:
        frequency_intensity_dict[frequency] += intensity
    else:
       
        frequency_intensity_dict[frequency] = intensity

result_frequencies_TIO2 = list(frequency_intensity_dict.keys())
result_intensities_TIO2 = list(frequency_intensity_dict.values())
plot(result_frequencies,result_intensities)


In [ ]:
from scipy.signal import argrelextrema
def g(wavenumb_sweep, intensity_max, wavenumber_max, σ):
    G = intensity_max / (σ *sqrt(2 * pi)) * exp(-(wavenumb_sweep-wavenumber_max)**2 / (2*σ**2))
    new_y=array(G)  
    return new_y
result_frequencies = list(frequency_intensity_dict.keys())
result_intensities = list(frequency_intensity_dict.values())      

wavenumb   = result_frequencies
max_int    = max(result_intensities)

norm_int = []
for i in result_intensities:
    norm_int.append(i/max_int)

### Gausian function to broaden peaks
pos_max    = argrelextrema(array(norm_int), np.greater)
x          = array(wavenumb)
all_curve = 0
σ = 20
for i in pos_max[0]:
    all_curve += g(x, norm_int[i], wavenumb[i],σ)      
broad_int = all_curve   

### Normalization

max_y = max(broad_int)
int_norm = []

for i in broad_int:
    int_norm.append((i/max_y))
plot(wavenumb,int_norm)



In [ ]:
import curses

def main(stdscr):
    # Configurar curses
    curses.curs_set(0)
    stdscr.clear()

    # Lista de elementos
    elementos = ["Elemento 1", "Elemento 2", "Elemento 3", "Otro elemento", "Ejemplo"]
    
    # Crear ventana para la búsqueda
    buscar_win = curses.newwin(1, curses.COLS - 2, 0, 1)
    buscar_win.addstr("Buscar: ")
    buscar_win.refresh()

    # Crear ventana para mostrar la lista de elementos
    lista_win = curses.newwin(curses.LINES - 2, curses.COLS - 2, 1, 1)

    # Crear el dropdown
    dropdown = curses.newwin(len(elementos), curses.COLS - 2, 2, 1)
    dropdown.keypad(True)
    curses.noecho()

    # Loop principal
    while True:
        lista_filtrada = [elem for elem in elementos if buscar_win.instr(0, 8).decode().lower() in elem.lower()]
        lista_win.clear()
        for i, elem in enumerate(lista_filtrada):
            lista_win.addstr(i, 0, elem)
        lista_win.refresh()

        ch = dropdown.getch()
        if ch == ord('q'):
            break

        if ch == curses.KEY_DOWN:
            dropdown.move(1, 0)
        elif ch == curses.KEY_UP:
            dropdown.move(-1, 0)

if __name__ == "__main__":
    curses.wrapper(main)



In [ ]:

freq_dft = [1.129780207231549269e+02 , 1.129780207231552680e+02 ,1.749879864729470853e+02 ,1.749879864729474264e+02 ,3.611532269867757350e+02 ,4.793102600546089320e+02 ,4.957302896450344178e+02 ,6.001592985713667758e+02 ,6.001592985713667758e+02] 
int_dft = [1.000000000000000000e+00,9.999999999999938938e-01,2.939221091892205324e-03,2.939221091892158921e-03,1.494269659327405464e-01,1.890540997545240437e-02,1.019312927046264333e-01,1.445200115409593922e-01,1.445200115409594477e-01]

def Gaus_norm( list_freq,list_int, sigma):
    ### Gausian function to broaden peaks
    int_dft_m      = list_int + list(zeros(len(wavenumb)))
    x_dft          = list_freq + wavenumb
    
    all= []
    for i in range(len(int_dft_m )):
        all.append(tuple((x_dft[i],int_dft_m[i])))
    
    all.sort()
    x= []
    y= []
    for i in all:
        x.append(round(i[0],1))
        y.append(i[1])
    
    frequencies = x
    intensities = y
    
    # Crear un diccionario para realizar un seguimiento de las sumas de intensidades por frecuencia
    frequency_intensity_dict = {}
    
    for frequency, intensity in zip(frequencies, intensities):
    
        if frequency in frequency_intensity_dict:
            frequency_intensity_dict[frequency] += intensity
        else:
           
            frequency_intensity_dict[frequency] = intensity
    
    result_frequencies_dft = list(frequency_intensity_dict.keys())
    result_intensities_dft = list(frequency_intensity_dict.values())    
    
    
    wavenumb_dft   = result_frequencies_dft
    max_int    = max(result_intensities_dft)
    
    norm_int = []
    for i in result_intensities_dft:
        norm_int.append(i/max_int)
    
    ### Gausian function to broaden peaks
    pos_max    = argrelextrema(array(norm_int), np.greater)
    
    x          = array(wavenumb)
    
    all_curve = 0
    σ = sigma
    for i in pos_max[0]:
        all_curve += g(x, norm_int[i], wavenumb[i],σ)      
    broad_int = all_curve   
    
    ### Normalization
    
    max_y = max(broad_int)
    int_norm_dft = []
    
    for i in broad_int:
        int_norm_dft.append((i/max_y))
    return wavenumb, int_norm_dft, result_intensities_dft



In [ ]:
# plot(result_frequencies, int_norm, label= "from Phonon Disp")
xlim(0,1000)
df2=RRUFF_text("anastase_532_exp.txt")._read_file()
plot(df2["wavenumbers"], df2["intensities_normalized"], label="Exp Anatase 532 nm")

freq_dft = [1.129780207231549269e+02 , 1.129780207231552680e+02 ,1.749879864729470853e+02 ,1.749879864729474264e+02 ,3.611532269867757350e+02 ,4.793102600546089320e+02 ,4.957302896450344178e+02 ,6.001592985713667758e+02 ,6.001592985713667758e+02] 
int_dft = [1.000000000000000000e+00,9.999999999999938938e-01,2.939221091892205324e-03,2.939221091892158921e-03,1.494269659327405464e-01,1.890540997545240437e-02,1.019312927046264333e-01,1.445200115409593922e-01,1.445200115409594477e-01]
freq_gn, int_gn = Gaus_norm(freq_dft,int_dft,20)

plot(freq_gn, int_gn, label= "DFT-Lorenzo")

# plot(df["Frequency"],df["Norm"], label="Exp Anatase")
xlabel(r"$Frequency[cm^{-1}]$")
ylabel("Intensity")
legend()

## Import RRUFF exp data

In [ ]:
from txt_raman_reader import RRUFF_text

In [ ]:
df=RRUFF_text("anastase_532_exp.txt")._read_file()

In [ ]:
import pandas as pd

In [ ]:
import functools

In [ ]:

# read by default 1st sheet of an excel file
df = pd.read_excel('anatase_exp.xlsx')
 
print(df)

In [ ]:
plot(float(array(df["Frequency"])),df["Norm"])

In [ ]:
freq=[]
for i in df["Frequency"]:
    freq.append(float(i))


# Using DATABASE

In [ ]:
from pylab import *

In [2]:
import json
 
# Opening JSON file
f = open('crd.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
data["4146"]["key_value_pairs"]["mpid"]

In [ ]:
data["1"].keys()

In [ ]:
data["1"]["data"].keys()

In [ ]:
data["1"]["data"]["lattice_angles"]

In [ ]:


chemical_formula   = data[ID]["data"]["chemical_formula"]
lattice_parameters = data[ID]["data"]["lattice_parameters"]
lattice_angles     = data[ID]["data"]["lattice_angles"]
cell_vectors       = array(data[ID]["cell"]["array"]["__ndarray__"][2]).reshape(3,3)
mpid               = data[ID]["key_value_pairs"]["mpid"]
density            = round(data[ID]["data"]["mass"]/data[ID]["data"]["volume"]*10**30*1.6726e-27,2)

print("The chemical formula of this material is: ", chemical_formula)
print("The lattice parameters of this material are, in the order (a,b,c) [A]: ", lattice_parameters)
print("The lattice angles of this material are, in the order (α,β,γ): ", lattice_angles)
print("The cell vectors of this material are, in matrix notation: ")
print(cell_vectors)
print(r"The density of this material in [kg/m^3]: ", density)
print("For more properties of the material you can consult it in the materials project webpage with the following ID: ", mpid)


In [ ]:
cell_vectors = array(data["1"]["cell"]["array"]["__ndarray__"][2]).reshape(3,3)
cell_vectors
mpid         = data["1"]["key_value_pairs"]["mpid"]


In [ ]:
data["4146"]["data"]["born"]

In [ ]:
print(data["4146"]["data"]["IRactive"]['__ndarray__'][2])
print(data["4146"]["data"]["Ramanactive"]['__ndarray__'][2])
print(((data["4146"]["key_value_pairs"]["frequencies_cm"])))
print(len(data["4146"]["data"]["eigenvec"][0]))

In [ ]:
ID="4146"
print(len(data[ID]["data"]["born"]))
print(len(data[ID]["data"]["eigenvec"]))
print(len(data[ID]["numbers"]['__ndarray__'][2]))
print((data[ID]["numbers"]['__ndarray__'][2]))

print(data[ID]["data"]["born"])
print((data[ID]["data"]["eigenvec"][0]))



In [ ]:
data["4146"]["numbers"]['__ndarray__'][2]

In [ ]:
data["4146"]["positions"]["__ndarray__"][2]

In [ ]:
eq_pos = reshape(data["4146"]["positions"]["__ndarray__"][2],\
                 (data["4146"]["positions"]["__ndarray__"][0][0],\
                  data["4146"]["positions"]["__ndarray__"][0][1]))
eq_pos[0][0]


In [ ]:
data["4146"]["positions"]["__ndarray__"][0][0]

In [ ]:
xyz = '''12
* (null), Energy   -1000.0000000
Ti    8.44346146e+00       0.00000000e+00       4.47994206e+00        0.3644459717394135         5.973428763633715e-05        0.13044106010002338
Ti    -1.22098741e-01      0.00000000e+00       4.43473766e+00        0.36444596702979787        5.973428763589211e-05        0.1304411344755619
Ti    1.83503573e+00      -2.22044605e-16       1.77994600e+00        0.36444597173941456        5.9734287635064e-05          0.1304410601000229
Ti    1.04005959e+01       0.00000000e+00       1.82515040e+00        0.36444596702979853        5.9734287634556656e-05       0.13044113447556266
O     4.53203947e-03       2.08166817e-16       2.31518849e+00        0.21070125145747576        3.4534838727218664e-05       0.07541266128956095
O     6.43306226e+00       0.00000000e+00       4.41253375e+00        0.21070108110973979        3.453483872759117e-05        0.07541290373989451
O     1.62407700e+00       2.22044605e-16       2.18768765e-02        0.2107014462466366         3.453483872736387e-05        0.07541313635197812
O     1.02739652e+01       0.00000000e+00       3.94469957e+00        0.21070125145747542        3.453483872757558e-05        0.0754126612895609
O     8.30906545e+00       0.00000000e+00       2.18423795e+00        0.21070093688908031        3.453483872721332e-05        0.07541376066561453
O     8.65442019e+00       0.00000000e+00       6.23801118e+00        0.21070144624663606        3.453483872743842e-05        0.0754131363519781
O     1.96943174e+00       0.00000000e+00       4.07565011e+00        0.21070093688908015        3.453483872753387e-05        0.07541376066561405
O     3.84543493e+00       0.00000000e+00       1.84735431e+00        0.2107010811097403         3.4534838727183766e-05       0.07541290373989427
'''

In [ ]:
view = py3Dmol.view(width=300, height=300)
view.addModel(xyz, "xyz", {'vibrate': {'frames':10,'amplitude':1}})
view.vibrate(10,1.,True,{"radius": 0.15,
                  "radiusRadio":0.8,
                  "mid":30, "color":"#8db600"})
view.setStyle({'sphere':{'scale':0.30},'stick':{'scale':0.15}})
view.setBackgroundColor('0xeeeeee')
view.animate({'loop': 'backAndForth'})
view.zoomTo()


In [ ]:
len(data["4146"]["data"]["eigenvec"])

In [ ]:
data["1"]["data"].keys()

In [ ]:
print(len(data["1"]["data"]["raman_tensors"]))
print(shape(data["4146"]["data"]["eigenvec"]))
print(len(data["1"]["data"]["born"]))


In [ ]:
data["1"]["data"]["born"]

In [ ]:
data["4146"]["data"]["eigenvec"]

In [ ]:

len(ints_ram_t)

In [ ]:
formulas =[]
compounds=[]
for i in range(1,5100):
    compounds.append((data["{0}".format(i)]["data"]["chemical_formula"],i))
    formulas.append(data["{0}".format(i)]["data"]["chemical_formula"])

In [ ]:
compounds.sort()
compounds

from dash import Dash, dcc, html, Input, Output,callback

app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(formulas, formulas[0], id='demo-dropdown'),
    html.Div(id='dd-output-container')
])


@callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    return f'You have selected {value}'


if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
for i in compounds:
    if i[0]=="Ti2O4":
        print(i)

In [ ]:
freq = data["4146"]["key_value_pairs"]["frequencies_cm"]
freq= freq.replace("[", " ")
freq=freq.replace("]", " ")
freq=freq.replace("\n", " ")
freq=freq.split()
freqs_db = [float(i) for i  in freq ] 
ints_ram_t = []
for i in data["4146"]["data"]["raman_tensors"]:
    ints_ram_t.append(intensity_raman(i))
    
int_db = data["4146"]["data"]["Ramanactive"]["__ndarray__"][2]

# freq_gn, int_gn, int_n = Gaus_norm(freqs_db,int_db,10)
# plot(freq_gn, int_n, label= "DFT-DB-1")

freq_gn, int_gn, int_n = Gaus_norm(freqs_db,ints_ram_t,10)
plot(freq_gn, int_gn, label= "DFT-DB")


df2=RRUFF_text("anastase_532_exp.txt")._read_file()
# plot(df2["wavenumbers"], df2["intensities_normalized"], label="Exp Anatase 532 nm")


df3=RRUFF_text("rutile_exp_dep.txt")._read_file()
plot(df3["wavenumbers"], df3["intensities_normalized"], label="Exp Rutile 532 nm")


freq_dft = [1.129780207231549269e+02 , 1.129780207231552680e+02 ,1.749879864729470853e+02 ,1.749879864729474264e+02 ,3.611532269867757350e+02 ,4.793102600546089320e+02 ,4.957302896450344178e+02 ,6.001592985713667758e+02 ,6.001592985713667758e+02] 
int_dft = [1.000000000000000000e+00,9.999999999999938938e-01,2.939221091892205324e-03,2.939221091892158921e-03,1.494269659327405464e-01,1.890540997545240437e-02,1.019312927046264333e-01,1.445200115409593922e-01,1.445200115409594477e-01]
freq_dft, int_dft, a = Gaus_norm(freq_dft,int_dft,20)

# plot(freq_dft, int_dft, label= "DFT-Lorenzo")

wavenum , int_norm, b = Gaus_norm(result_frequencies_TIO2, result_intensities_TIO2,20)
plot(wavenumb,int_norm, label="Phonon dis")
# plot(df["Frequency"],df["Norm"], label="Exp Anatase")
xlabel(r"$Frequency[cm^{-1}]$")
ylabel("Intensity")

# plot(freq_db_all, new_int_all, label=r"$Combined - \Gamma point$")
legend()

ylim(-0.01,1)
xlim(-10,1000)


In [ ]:
freqs_db_r =[round(i,1) for i in freqs_db]
print(freqs_db_r,int_db)

In [ ]:
import json
 
# Opening JSON file
f = open('crd.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
ID="52"


In [1]:
from quetzalcoatl_1 import Animator_data, spectra_data, solids_spectra

In [2]:
ID ="78"
Animator_data(ID)


In [4]:

sigma_IR=0.1
sigma_Raman=0.1

freq_gn_ir, int_gn_ir, int_n_ir , freq_gn_raman, int_gn_raman, int_n_raman= spectra_data(ID, sigma_IR, sigma_Raman)
solids_spectra("readme_solids.txt", freq_gn_ir, int_gn_ir, freq_gn_raman, int_gn_raman)


The chemical formula of this material is:  Ba8Cu4I4O8
The lattice parameters of this material are, in the order (a,b,c) [A]:  [10.95891831  4.38039471 13.93294818]
The lattice angles of this material are, in the order (α,β,γ):  [90.         91.18449953 90.        ]
The cell vectors of this material are, in matrix notation: 
[[10.95890004  0.          0.02001305]
 [ 0.          4.38039471  0.        ]
 [-0.3134594   0.         13.92942167]]
The density of this material in [kg/m^3]:  4973.55
For more properties of the material you can consult it in the materials project webpage with the following ID:  mp-561179


interactive(children=(Dropdown(description='Normal mode:', options=((0.0, 0), (0.0, 1), (0.0, 2), (23.37, 3), …